In [11]:
#Reference => sentdex tutorial on Youtube
!pip install gym #already installed

In [4]:
import gym
env = gym.make('CartPole-v0') #for discription https://github.com/openai/gym/wiki/CartPole-v0
env.reset()

array([ 0.01990369,  0.01409229,  0.02234002, -0.00967025])

In [5]:
import random
done = False

while not done:
  action = (int(random.random()*10))%2 #action 0->push cart left; action 1->push cart right
  state, reward, done, _ = env.step(action)# state is a tuple of (cart position, cart velocity, pole angle, pole velocity at tip)
  print(state)  
  env.render()

env.close()

#the cart need to balance the pole as much as possible

[ 0.02018554  0.20888685  0.02214662 -0.29522176]
[0.02436328 0.01345628 0.01624218 0.00436278]
[ 0.0246324  -0.18189479  0.01632944  0.30212573]
[0.02099451 0.01299067 0.02237195 0.01463716]
[ 0.02125432  0.20778475  0.02266469 -0.27090404]
[0.02541001 0.01234683 0.01724661 0.02884039]
[ 0.02565695  0.20721726  0.01782342 -0.25835153]
[ 0.0298013   0.40208029  0.01265639 -0.54535983]
[ 0.0378429   0.59702213  0.00174919 -0.8340283 ]
[ 0.04978334  0.40187633 -0.01493137 -0.54079578]
[ 0.05782087  0.59720494 -0.02574729 -0.8381457 ]
[ 0.06976497  0.40244388 -0.0425102  -0.5536698 ]
[ 0.07781385  0.59813618 -0.0535836  -0.8594373 ]
[ 0.08977657  0.79394544 -0.07077234 -1.16847567]
[ 0.10565548  0.59981192 -0.09414186 -0.89879406]
[ 0.11765172  0.79607505 -0.11211774 -1.219522  ]
[ 0.13357322  0.6025625  -0.13650818 -0.96396926]
[ 0.14562447  0.40951227 -0.15578756 -0.71709665]
[ 0.15381471  0.60640779 -0.1701295  -1.05447972]
[ 0.16594287  0.80332614 -0.19121909 -1.39536996]
[ 0.18200939

## Implementing Q Table

We will be implementing SARSAMAX

![](SARSAMAX.png)

- State contains continous values
- Q Table will contains prohibitively large number of state values
- We need to **dicretize them** in order to create Q Table
- Initially Q Table will contain random values

In [6]:
#display observation space low and high values
print(env.observation_space.high)#[position,cart_velocity,pole_angle,pole_velocity]
print(env.observation_space.low)#[position,cart_velocity,pole_angle,pole_velocity]
print(env.action_space.n)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
2


In [7]:
#Reference 1 for discretizizng -> https://medium.com/@tuzzer/cart-pole-balancing-with-q-learning-b54c6068d947 
#Reference 2 for discretizing -> https://github.com/deepakkavoor/cartpole-rl/blob/master/cartpole-q_learning.py
import math
env.observation_space.high[1]= 0.5
env.observation_space.low[1]= -0.5
env.observation_space.high[3]= math.radians(50)
env.observation_space.low[3]= -math.radians(50)

## Now we will implement SARSAMAX algorithm to fill the Q Table

In [19]:
import numpy as np
LEARNING_RATE = 0.1 #hyperparameter 
DISCOUNT = 0.95 #hyperparameter; how important are future rewards  
EPISODES = 20000 #hyperparameter
RENDER_EVERY = 1000



#Reference for discretizing=> https://mc.ai/openai-gyms-cart-pole-balancing-using-q-learning/
buckets=(1, 1, 6, 12,) #hyperparameter -> dimensions of Q Table (position,cart_velocity,pole_angle,pole_velocity)
q_table = np.zeros(buckets + (env.action_space.n,))
print(q_table.shape)



#a helper function to discretize continous states as returned by the environment 
def get_discrete_state(state):
    ratios = [(state[i] + abs(env.observation_space.low[i])) / (env.observation_space.high[i] - env.observation_space.low[i]) for i in range(len(state))]
    new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(state))]
    new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(state))]
    return tuple(new_obs)
    
    
    


(1, 1, 6, 12, 2)


In [20]:
# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

In [21]:
for episode in range(EPISODES):
    done = False
    discrete_state = get_discrete_state(env.reset()) #we always start with initial discrete state'
    if episode % RENDER_EVERY == 0:
        render=True
        print(episode)
    else:
        render=False
    steps_per_episode = 0
    while not done:
        
        if render:
            env.render()
            steps_per_episode+=1
            print('Step Count'+str(steps_per_episode))
        
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)
        
        state, reward, done, _ = env.step(action)# state is a tuple of (position,cart_velocity,pole_angle,pole_velocity)
        new_discrete_state = get_discrete_state(state)
        if not done:
            q_st_at=q_table[discrete_state+(action,)]
            q_st_plus_1_a = np.max(q_table[new_discrete_state])
            new_q_st_at = q_st_at + LEARNING_RATE * (reward + DISCOUNT * q_st_plus_1_a - q_st_at)
            q_table[discrete_state+(action,)] = new_q_st_at
        discrete_state = new_discrete_state
     # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value
env.close()

0
Step Count1
Step Count2
Step Count3
Step Count4
Step Count5
Step Count6
Step Count7
Step Count8
Step Count9
Step Count10
Step Count11
Step Count12
Step Count13
Step Count14
Step Count15
1000
Step Count1
Step Count2
Step Count3
Step Count4
Step Count5
Step Count6
Step Count7
Step Count8
Step Count9
Step Count10
Step Count11
Step Count12
Step Count13
Step Count14
Step Count15
2000
Step Count1
Step Count2
Step Count3
Step Count4
Step Count5
Step Count6
Step Count7
Step Count8
Step Count9
Step Count10
Step Count11
Step Count12
Step Count13
Step Count14
Step Count15
Step Count16
Step Count17
3000
Step Count1
Step Count2
Step Count3
Step Count4
Step Count5
Step Count6
Step Count7
Step Count8
Step Count9
Step Count10
Step Count11
Step Count12
Step Count13
Step Count14
Step Count15
Step Count16
Step Count17
Step Count18
Step Count19
Step Count20
Step Count21
Step Count22
Step Count23
Step Count24
Step Count25
Step Count26
Step Count27
Step Count28
Step Count29
Step Count30
Step Count31
Step 

Step Count79
Step Count80
Step Count81
Step Count82
Step Count83
Step Count84
Step Count85
Step Count86
Step Count87
Step Count88
Step Count89
Step Count90
Step Count91
Step Count92
Step Count93
Step Count94
Step Count95
Step Count96
Step Count97
Step Count98
Step Count99
Step Count100
Step Count101
Step Count102
Step Count103
Step Count104
Step Count105
Step Count106
Step Count107
Step Count108
Step Count109
Step Count110
Step Count111
Step Count112
Step Count113
Step Count114
Step Count115
Step Count116
Step Count117
Step Count118
Step Count119
Step Count120
Step Count121
Step Count122
Step Count123
Step Count124
Step Count125
Step Count126
Step Count127
Step Count128
Step Count129
Step Count130
Step Count131
Step Count132
Step Count133
Step Count134
Step Count135
Step Count136
Step Count137
Step Count138
Step Count139
Step Count140
Step Count141
Step Count142
Step Count143
Step Count144
Step Count145
Step Count146
Step Count147
Step Count148
Step Count149
Step Count150
Step Count151

Step Count92
Step Count93
Step Count94
Step Count95
Step Count96
Step Count97
Step Count98
Step Count99
Step Count100
Step Count101
Step Count102
Step Count103
Step Count104
Step Count105
Step Count106
Step Count107
Step Count108
Step Count109
Step Count110
Step Count111
Step Count112
Step Count113
Step Count114
Step Count115
Step Count116
Step Count117
Step Count118
Step Count119
Step Count120
Step Count121
Step Count122
Step Count123
Step Count124
Step Count125
Step Count126
Step Count127
Step Count128
Step Count129
Step Count130
Step Count131
Step Count132
Step Count133
Step Count134
Step Count135
Step Count136
Step Count137
Step Count138
Step Count139
Step Count140
Step Count141
Step Count142
Step Count143
Step Count144
Step Count145
Step Count146
Step Count147
Step Count148
Step Count149
Step Count150
Step Count151
Step Count152
Step Count153
Step Count154
Step Count155
Step Count156
Step Count157
Step Count158
Step Count159
Step Count160
Step Count161
Step Count162
Step Count163


Step Count105
Step Count106
Step Count107
Step Count108
Step Count109
Step Count110
Step Count111
Step Count112
Step Count113
Step Count114
Step Count115
Step Count116
Step Count117
Step Count118
Step Count119
Step Count120
Step Count121
Step Count122
Step Count123
Step Count124
Step Count125
Step Count126
Step Count127
Step Count128
Step Count129
Step Count130
Step Count131
Step Count132
Step Count133
Step Count134
Step Count135
Step Count136
Step Count137
Step Count138
Step Count139
Step Count140
Step Count141
Step Count142
Step Count143
Step Count144
Step Count145
Step Count146
Step Count147
Step Count148
Step Count149
Step Count150
Step Count151
Step Count152
Step Count153
Step Count154
Step Count155
Step Count156
Step Count157
Step Count158
Step Count159
Step Count160
Step Count161
Step Count162
Step Count163
Step Count164
Step Count165
Step Count166
Step Count167
Step Count168
Step Count169
Step Count170
Step Count171
Step Count172
Step Count173
Step Count174
Step Count175
Step C

Step Count118
Step Count119
Step Count120
Step Count121
Step Count122
Step Count123
Step Count124
Step Count125
Step Count126
Step Count127
Step Count128
Step Count129
Step Count130
Step Count131
Step Count132
Step Count133
Step Count134
Step Count135
Step Count136
Step Count137
Step Count138
Step Count139
Step Count140
Step Count141
Step Count142
Step Count143
Step Count144
Step Count145
Step Count146
Step Count147
Step Count148
Step Count149
Step Count150
Step Count151
Step Count152
Step Count153
Step Count154
Step Count155
Step Count156
Step Count157
Step Count158
Step Count159
Step Count160
Step Count161
Step Count162
Step Count163
Step Count164
Step Count165
Step Count166
Step Count167
Step Count168
Step Count169
Step Count170
Step Count171
Step Count172
Step Count173
Step Count174
Step Count175
Step Count176
Step Count177
Step Count178
Step Count179
Step Count180
Step Count181
Step Count182
Step Count183
Step Count184
Step Count185
Step Count186
Step Count187
Step Count188
Step C

## Sample Run of Agent

In [22]:

done = False
discrete_state = get_discrete_state(env.reset()) #we always start with initial discrete state'
steps=0
while not done:
    steps+=1
    print(steps)
    env.render()
        
    action = np.argmax(q_table[discrete_state])
        
    state, reward, done, _ = env.step(action)# state is a tuple of (position,cart_velocity,pole_angle,pole_velocity)

    discrete_state=get_discrete_state(state)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
